In [16]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-03-06 05:14:43--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-03-06 05:14:44--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-03-06 05:14:45--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [28]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)


Spark NLP version 3.4.1
Apache Spark version: 3.0.3


In [18]:
from sparknlp.pretrained import PretrainedPipeline

In [19]:
# Download a pre-trained pipeline by name and language
language_detector_pipeline = PretrainedPipeline('detect_language_21', lang='xx')

# Depending on the language (how similar the characters are), the LanguageDetectorDL works
# best with text longer than 140 characters
language_detector_pipeline.annotate("«Нападение на 13-й участок»")

detect_language_21 download started this may take some time.
Approx size to download 7.7 MB
[OK!]


{'document': ['«Нападение на 13-й участок»'],
 'language': ['bg'],
 'sentence': ['«Нападение на 13-й участок»']}

In [20]:
from sparknlp.base import *
from sparknlp.annotator import *

In [21]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

language_detector = LanguageDetectorDL.pretrained("ld_wiki_tatoeba_cnn_21")\
.setInputCols(["document"])\
.setOutputCol("lang")\
.setThreshold(0.8)\
.setCoalesceSentences(True)

languagePipeline = Pipeline(stages=[
 documentAssembler, 
 language_detector
])

ld_wiki_tatoeba_cnn_21 download started this may take some time.
Approximate size to download 7.1 MB
[OK!]


In [54]:
# test_df = spark.createDataFrame([
#   ['Spark NLP is an open-source text processing library for advanced natural language processing for the Python, Java and Scala programming languages.'], 
#   ['Spark NLP est une bibliothèque de traitement de texte open source pour le traitement avancé du langage naturel pour les langages de programmation Python, Java et Scala.']]
# ).toDF("text")
# test_df.show()

path = "/content/drive/MyDrive/big-data-analytic/health.csv"
test_df = spark.read.csv(path).toDF("text")
test_df.show()

results = languagePipeline.fit(test_df).transform(test_df)

+--------------------+
|                text|
+--------------------+
|                text|
|                I...|
|                B...|
|                T...|
|                null|
|                M...|
|                T...|
|                I...|
|                T...|
|Anemias continue ...|
|                D...|
+--------------------+



In [55]:
results.select("lang.result").show()

+---------+
|   result|
+---------+
|[Unknown]|
|     [en]|
|     [en]|
|     [en]|
|       []|
|     [en]|
|     [en]|
|     [en]|
|     [en]|
|     [en]|
|     [en]|
+---------+



In [56]:
# probabilities for other languages
results.select("lang.metadata").show(2, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|metadata                                                                                                                                                                                                                                                                                                                                                                                                                |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------